In [10]:
import pandas as pd
import numpy as np
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized_v2

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [11]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbols as column names.
  The function first calculates symbols' drawdown, Ulcer-Index, max-drawdown, std(returns),
  std(returns)/Ulcer-Index, CAGR, CAGR/std(returns), CAGR/Ulcer-Index. Then it calculates and returns 
  the group's (i.e. all the symbols) mean/standard-deviation for 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI'.

  Args:
    df_close(dataframe): dataframe of symbols' close with
      DatetimeIndex e.g. (['2016-12-19', ... '2016-12-22']), symbols as
      column names, and symbols' close as column values.

  Return:
    df_perf(dataframe): dataframe with columns symbol, first date, last date, Year, CAGR,	UI,
    	return_std/UI, CAGR/return_std, CAGR/UI
    grp_retnStd_d_UI_mean_div_std(float): (std(returns)/Ulcer-Index).mean / (std(returns)/Ulcer-Index).std
    grp_CAGR_d_retnStd_mean_div_std(float): (CAGR/std(returns)).mean / (CAGR/std(returns)).std
    grp_CAGR_d_UI_mean_div_std(float): (CAGR/Ulcer_Index).mean / (CAGR/Ulcer_Index).std
  '''

  from myUtils import symb_perf_stats_vectorized_v2

  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, returns_std_div_UI, CAGR, CAGR_div_returns_std, CAGR_div_UI = \
      symb_perf_stats_vectorized_v2(df_close)

  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], returns_std_div_UI[symbol], CAGR_div_returns_std[symbol], CAGR_div_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)

  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'return_std/UI', 'CAGR/return_std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI']
  grp_CAGR_d_UI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGR_d_UI_std  = df_perf['CAGR/UI'].std()
  grp_CAGR_d_UI_mean_div_std = grp_CAGR_d_UI_mean / grp_CAGR_d_UI_std 

  grp_CAGR_d_retnStd_mean = df_perf['CAGR/return_std'].mean()
  grp_CAGR_d_retnStd_std = df_perf['CAGR/return_std'].std()
  grp_CAGR_d_retnStd_mean_div_std = grp_CAGR_d_retnStd_mean / grp_CAGR_d_retnStd_std

  grp_retnStd_d_UI_mean = df_perf['return_std/UI'].mean()
  grp_retnStd_d_UI_std = df_perf['return_std/UI'].std()
  grp_retnStd_d_UI_mean_div_std = grp_retnStd_d_UI_mean / grp_retnStd_d_UI_std

  return df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std

In [12]:
df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std = perf_eval(df_c)
print(f'retnStd/UI   grp_mean/grp_std: {grp_retnStd_d_UI_mean_div_std:>10.6f}')
print(f'CAGR/retnStd grp_mean/grp_std: {grp_CAGR_d_retnStd_mean_div_std:>10.6f}')
print(f'CAGR/UI      grp_mean/grp_std: {grp_CAGR_d_UI_mean_div_std:>10.6f}')
df_perf.head()

retnStd/UI   grp_mean/grp_std:   2.014990
CAGR/retnStd grp_mean/grp_std:  -0.294383
CAGR/UI      grp_mean/grp_std:   0.064977


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,A,2021-12-22,2022-12-21,1.0,-0.036540,0.182503,0.121852,-1.643115,-0.200217
1,AA,2021-12-22,2022-12-21,1.0,-0.237865,0.404067,0.102902,-5.720764,-0.588678
2,AAL,2021-12-22,2022-12-21,1.0,-0.286418,0.253589,0.139572,-8.092294,-1.129459
3,AAON,2021-12-22,2022-12-21,1.0,-0.012630,0.266593,0.092403,-0.512723,-0.047377
4,AAP,2021-12-22,2022-12-21,1.0,-0.366756,0.218368,0.107576,-15.612478,-1.679532


In [13]:
my_symbols = ['AE', 'ABMD', 'RMBS', 'WNC']
my_cols = ['symbol', 'return_std/UI',	'CAGR/return_std', 'CAGR/UI']
# my_cols = ['return_std/UI',	'CAGR/return_std', 'CAGR/UI']

In [14]:
# %%timeit -n 50 -r 20
df_temp = df_c[my_symbols]
df_perf1, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std = perf_eval(df_temp)
print(f'return_std/UI   grp_mean/grp_std: {grp_mean_div_grp_std[0]:.6f}')
print(f'CAGR/return_std grp_mean/grp_std: {grp_mean_div_grp_std[1]:.6f}')
print(f'CAGR/UI         grp_mean/grp_std: {grp_mean_div_grp_std[2]:.6f}')
df_perf1

return_std/UI   grp_mean/grp_std: 4.364072
CAGR/return_std grp_mean/grp_std: 1.386309
CAGR/UI         grp_mean/grp_std: 1.115244


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,AE,2021-12-22,2022-12-21,1.0,0.546331,0.135996,0.184570,21.765435,4.017253
1,ABMD,2021-12-22,2022-12-21,1.0,0.109552,0.225783,0.186714,2.598659,0.485207
2,RMBS,2021-12-22,2022-12-21,1.0,0.271478,0.202212,0.126886,10.580702,1.342542
3,WNC,2021-12-22,2022-12-21,1.0,0.271491,0.243077,0.121657,9.180664,1.116895


In [15]:
# %%timeit -n 50 -r 20
# select rows in my_symbols and columns in my_cols from df_perf
_df = df_perf.loc[df_perf['symbol'].isin(my_symbols)][my_cols]
# skip symbol in my_cols[0]
grp_mean_div_grp_std = _df[my_cols[1:]].mean(numeric_only=True) / _df[my_cols[1:]].std(numeric_only=True)
print(f'{my_cols[1]}   grp_mean/grp_std: {grp_mean_div_grp_std[0]:.6f}')
print(f'{my_cols[2]} grp_mean/grp_std: {grp_mean_div_grp_std[1]:.6f}')
print(f'{my_cols[3]}         grp_mean/grp_std: {grp_mean_div_grp_std[2]:.6f}')
_df

return_std/UI   grp_mean/grp_std: 4.364072
CAGR/return_std grp_mean/grp_std: 1.386309
CAGR/UI         grp_mean/grp_std: 1.115244


,symbol,return_std/UI,CAGR/return_std,CAGR/UI
15,ABMD,0.186714,2.598659,0.485207
39,AE,0.184570,21.765435,4.017253
1761,RMBS,0.126886,10.580702,1.342542
2243,WNC,0.121657,9.180664,1.116895


In [16]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: nan
grp_CAGRStd_mean_std: nan
grp_CAGRUI_mean_std: nan


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,SPY,2021-12-22,2022-12-21,1.0,-0.174175,0.152872,0.099621,-11.436923,-1.139353
